<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/Teste_ABCOptim_jun_12_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""ABCOptim_jun_10_2019.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1KxLGSILwQZQySSynZeoDYtyG4TLz_IRh
"""

import numpy as np
from random import randint
#import Function
#import Enxame

''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''


def Fun(x):
    
    summ=0
    for i in range(len(x)):
        new=x[i]*np.sin((abs(x[i]))**0.5)
        summ=summ+new
#        print(summ)
    return (418.9829*len(x)-summ)  

def FOBJ(X,Fun):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X,Fun):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X,Fun))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x,Fun):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x,Fun)
    fit_ref=max(fit)
    
    for i in range(rows):
        prob[i] = 0.9*fit[i]/fit_ref+0.1

    return prob

'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest,Fun):
    ycal=FOBJ(x,Fun)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST

'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN,trial,Fun) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo,Fun)
    YV=FOBJ(V,Fun)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
          trial[i]=0
          xo[i,]=np.copy(V[i,]) #MODIFICADO JUN 12
        else:
          trial[i]=trial[i]+1
            

    return xo,trial  #MODIFICADO JUN 12

'''
onlooker bee phase
'''
def OnlookerBee(xo,x,trial,MAX,MIN,Fun): #OnlookerBee(xo,x_Employed)
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x,Fun)
    prob_ref=max(prob)
    #best=np.argmax(prob)
    
    for i in range(rows):
      prob_ref=np.random.uniform(low=0, high=1.0, size=None)
      rd=randint(0, (rows-1))
      if(prob[i] > prob_ref):  
        fi=np.random.uniform(low=-1.0, high=1.0, size=None)
        Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
        xo[i,]=np.copy(Xmi)
        for j in range(cols):
          if (xo[i,j]>MAX[j]):
            xo[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
          if (xo[i,j]<MIN[j]):
            xo[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
    
    YCAL=FOBJ(x,Fun)
    YV=FOBJ(xo,Fun)
        
    for i in range(rows):
      if(YV[i]<YCAL[i]):
        trial[i]=0
        x[i,]=np.copy(xo[i,])
      else:
        trial[i]=trial[i]+1
    return xo,trial

'''
scout bee phase
'''
def ScoutBee(trial,x,ntrail,MAX,MIN):
#def ScoutBee(x,trial,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
      if(trial[i]>= ntrail):
        trial[i]=0
        for j in range(NPAR):
          x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x,trial

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search
'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN,Fun):

    trial=np.zeros(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    xbest=GetBest(xo,xbest,Fun)
    
    for i in range(ITE):
      x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para employed bee
      xo,trial=EmployedBee(xo,x,MAX,MIN,trial,Fun)
      xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
      x=Enxame(PAR,NPAR,MAX,MIN)  # comparativo para OnlookerBee
      xo,trial=OnlookerBee(xo,x,trial,MAX,MIN,Fun)
      xbest=GetBest(xo,xbest,Fun) 
      #xbest=GetBest(xo,xbest)
      # x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para ScoutBee
      xo,trial=ScoutBee(trial,x,ntrail,MAX,MIN) #MODIFICADO JUN 12
      xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
    x=np.copy(xo)
    xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
    
    return xbest, x,trial

#'''
    
dim=3
#Fun=Function.Schwefel
MAX=np.repeat(500,dim)
MIN=np.repeat(-500,dim)

NPAR=175 #PARTICULAS (fontes de comida)
ITE=300 #ITERACOES
PAR=len(MAX)
ntrail=75 #numero de buscas ate abandonar uma fonte de alimento

for i in range(20):
    print("resolucao",i+1," ")
    xbest, x,trial =ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN,Fun)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")

#print(trial)
#'''

resolucao 1  
vetor [-304.09791563  401.50726086 -316.58595215] funcao 308.5942581949936 

resolucao 2  
vetor [ 437.2900448   396.06115461 -500.        ] funcao 346.86381781843545 

resolucao 3  
vetor [433.89646507 457.22789295 409.74026566] funcao 194.46118718046637 

resolucao 4  
vetor [-303.98517431 -290.65531792  415.50774767] funcao 258.4491996007448 

resolucao 5  
vetor [-290.56085827  420.67218015  415.1963389 ] funcao 140.4670158633603 

resolucao 6  
vetor [421.21754063 373.58675325 434.27233323] funcao 269.1119935278657 

resolucao 7  
vetor [-286.8881357   414.94814989  425.69389668] funcao 155.98653916174817 

resolucao 8  
vetor [441.84911492 398.12487539 437.30817867] funcao 151.26574632184543 

resolucao 9  
vetor [ 430.47777765  214.32635237 -290.04535542] funcao 380.3885318726898 

resolucao 10  
vetor [ 395.96654158 -317.73809404 -313.14394365] funcao 355.8428613058717 

resolucao 11  
vetor [399.82054568 400.99183282 455.96485624] funcao 251.1440256840359 

resol

In [2]:
FOBJ(x,Fun)

array([ 868.03711151,  830.61808526,  354.91247767, 1263.10185124,
        823.65853577, 1195.43965315, 1539.65958823,  610.66931511,
        737.8250209 , 1087.92720327, 1015.31864344, 1211.02826871,
       1230.77687344, 1226.05116365,  782.37714253,  963.68864542,
        836.6962323 , 1297.8783745 ,  893.57348865,  890.37922656,
       1107.93698484, 1177.9898834 ,  973.33771151, 1437.53785853,
       1063.0062934 ,  693.13359521, 1040.38135785, 1522.01706557,
        727.5388017 ,  887.80971879, 1254.81716837, 1258.45323193,
        893.00973064,  972.47055418,  705.93593679, 1076.96585126,
       1621.76888082, 1035.36386659,  873.81120985,  997.520965  ,
        976.5698733 ,  640.30104697,  819.55427993,  779.84146632,
       1008.37486825, 1220.36213931, 1071.26707308,  690.7472228 ,
        792.09219518, 1412.28079625,  942.16977197,  734.64905663,
       1247.40896649,  736.82691873, 1023.06426815,  665.9439898 ,
        895.0358119 ,  998.82349766,  501.56794035,  730.29812

In [3]:
min(FOBJ(x,Fun))

229.63573702390704

In [4]:
Fun(xbest)

229.63573702390704